# Part Two:

1. Go to the Gambit website and read the documentation on the Python API.

2. Then, build a notebook to study a 2x2 risky coordination game (stage hunt is one) with agents having utility payoffs over dollar values in the normal form game.

3. Explore different risk aversion parameters to see if risk aversion will affect the game's play. Why or why not?

## Stag-Hunt Coordination Game Setup

In [1]:
import pygambit
import utility_functions as uf
import copy

# Setup PyGambit
game = pygambit.Game.new_table([2,2])
game.title = "Stag-Hare Game"
game.players[0].label = "Player 1"
game.players[1].label = "Player 2"

# Setup Payoffs
payoffs = {'st_st' : 10, 'hr_hr' : 8, 'st_hr' :1 , 'hr_st' : 8}

x = list((payoffs.values()))

# Payoffs for Utility Functions
lin_payoff = []
cara_payoff = []
crra_payoff = []
exp_payoff = []
hara_payoff = []
for payoff in range(len(x)):
    lin_payoff.append(round(uf.linear_utility(m = x[payoff]),2))
    cara_payoff.append(round(uf.cara_utility(m = x[payoff]),2))
    crra_payoff.append(round(uf.crra_utility(m = x[payoff]),2))
    exp_payoff.append(round(uf.exponential_utility(m = x[payoff]),2))
    hara_payoff.append(round(uf.hara_utility(m = x[payoff]),2))

all_payoff = [lin_payoff,cara_payoff,crra_payoff,exp_payoff,hara_payoff]
all_payoff_label = ["Linear Utility","CARA Utility", "CRRA Utility", "Exponential Utility", "HARA Utility"]

# Setting up Strategies
for player in range(len(game.players)):
    game.players[player].strategies[0].label = "Stag"
    game.players[player].strategies[1].label = "Hare"

# Neat Printing of Game by Utility Functions
for label, utilfn in zip(all_payoff_label,all_payoff):
    print(f"------------{label}:{utilfn}-------------")
    for outcome in game.contingencies:
        p1_choice = game.players[0].strategies[outcome[0]].label
        p2_choice = game.players[1].strategies[outcome[1]].label
        if p1_choice == p2_choice:
            game[outcome]["Player 1"] = utilfn[outcome[0]]
            game[outcome]["Player 2"] = utilfn[outcome[1]]
        else:
            game[outcome]["Player 1"] = utilfn[outcome[0] + 2]
            game[outcome]["Player 2"] = utilfn[outcome[1] + 2]
        print(p1_choice, p2_choice)
        print(game[outcome]["Player 1"],game[outcome]["Player 2"])

    pure_nash = pygambit.nash.enummixed_solve(game)

    # Enumerating over all equilibria
    for eq_idx, eq in enumerate(pure_nash.equilibria):
        print("---------------------------------")
        print(f"Equilibrium {eq_idx + 1}:")

        # Printing  strategy labels for each player with the corresponding probability for Mixed Outcomes
        for player in range(len(game.players)):
            print(f"Player {player + 1}:")
            strategy_profile = eq[game.players[player]]  # get the player's strategy profile
            for strategy in game.players[player].strategies:
                prob = float(strategy_profile[strategy])  # get the probability of each strategy
                print(f"  Strategy {strategy.label}: {prob:.2f}")

------------Linear Utility:[10.0, 8.0, 1.0, 8.0]-------------
Stag Stag
10.0 10.0
Stag Hare
1.0 8.0
Hare Stag
8.0 1.0
Hare Hare
8.0 8.0
---------------------------------
Equilibrium 1:
Player 1:
  Strategy Stag: 1.00
  Strategy Hare: 0.00
Player 2:
  Strategy Stag: 1.00
  Strategy Hare: 0.00
---------------------------------
Equilibrium 2:
Player 1:
  Strategy Stag: 0.78
  Strategy Hare: 0.22
Player 2:
  Strategy Stag: 0.78
  Strategy Hare: 0.22
---------------------------------
Equilibrium 3:
Player 1:
  Strategy Stag: 0.00
  Strategy Hare: 1.00
Player 2:
  Strategy Stag: 0.00
  Strategy Hare: 1.00
------------CARA Utility:[0.05, 0.04, 0.0, 0.04]-------------
Stag Stag
0.05 0.05
Stag Hare
0.0 0.04
Hare Stag
0.04 0.0
Hare Hare
0.04 0.04
---------------------------------
Equilibrium 1:
Player 1:
  Strategy Stag: 1.00
  Strategy Hare: 0.00
Player 2:
  Strategy Stag: 1.00
  Strategy Hare: 0.00
---------------------------------
Equilibrium 2:
Player 1:
  Strategy Stag: 0.80
  Strategy Hare

*Note: The Cell Output can be expanded*

## Question 3
Using different utility functions to look at alternative risk parameters allows us to see the changes in the structure of payoffs for the game. Pure strategies like (Stag, Stag) and (Hare, Hare) stay the same no matter the utility function used, but mixed strategies change somewhat between different utility functions. Risk aversion alters how players evaluate payoffs. Under linear utility, players should be risk neutral, but under CARA or CRRA, they should be risk averse.

The payoffs are transformed by the utility functions, which take the initial payoff as input to provide a calculated utility of the payoff. This transformation for each payoff kept the ordering of the payoffs the same.